In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk  
from itertools import repeat
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.optimizers import Adam
import keras
from tensorflow.keras import backend
from keras import backend as K
from keras import regularizers
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import words
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import sys
import gc
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

Using TensorFlow backend.


In [2]:
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)
       

In [4]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size))) 

                    stop_words:  8.2 KiB
                    Sequential:  1.0 KiB
                         Dense:  1.0 KiB
                          LSTM:  1.0 KiB
                          Adam:  1.0 KiB
               CountVectorizer:  1.0 KiB
                          tqdm:  1.0 KiB
                 PorterStemmer:  1.0 KiB
             WordNetLemmatizer:  1.0 KiB
                          _iii:  923.0 B


In [5]:
labels = []
documents_train = []
documents_test  = []
Y_train = []
Y_test =  []
directory = "/home/daniela/Projeto/Fixing code/PredadorAllData/PredadorAllDataNgrmas_alldata/"

trenamento = 0
ii = 0
for index, name in enumerate(sorted(os.listdir(directory))): #Already sorted
    trenamento += 1
    #if name in tittles_matlab:
        #print(name)
    ii += 1
    f=open("/home/daniela/Projeto/Fixing code/PredadorAllData/PredadorAllDataNgrmas_alldata/"+name, encoding ="utf-8")#"rb"
    #f=open("/home/daniela/Documents/Copia de csa_Predador/Copia de csa_Predador/TMGresults/sample2/"+name,"rb")#"rb"
    document = (f.read()) 

    if "trrainninng" in name:
        documents_train.append(str(document))
        if "posi" in name:
            labels.append(1)
            Y_train.append(1)

        else:
            labels.append(0)
            Y_train.append(0)

    elif "teestiinng" in name:
        documents_test.append(str(document))
        if "posi" in name:
            labels.append(1)
            Y_test.append(1)

        else:
            labels.append(0)
            Y_test.append(0)

print("Num of Labels: ",len(labels))
print("Num of Train docs: ",len(documents_train))
print("Num of Train Labels: ",len(Y_train))
print("Num of Test docs: ",len(documents_test))
print("Num of Test Labels: ",len(Y_test))
#'''
print("GOL: ",ii," Real: ",trenamento)

Num of Labels:  222055
Num of Train docs:  66927
Num of Train Labels:  66927
Num of Test docs:  155128
Num of Test Labels:  155128
GOL:  222055  Real:  222055


In [6]:
Y_train = np.asarray(Y_train)
Y_test = np.asarray(Y_test)
Y_test_reduce = Y_test


posi_train = np.argwhere(Y_train==1)
posi_test  = np.argwhere(Y_test==1)
nega_train = np.argwhere(Y_train==0)
nega_test  = np.argwhere(Y_test==0)
print("Predadors: ",len(posi_train)," No Predadors: ", len(nega_train))
print("Predadors: ",len(posi_test)," No Predadors: ", len(nega_test))

Predadors:  2016  No Predadors:  64911
Predadors:  3737  No Predadors:  151391


In [7]:
print('There are: ',len(documents_train),'   Train labels')
print('There are: ',len(documents_test),'  Test  labels')

There are:  66927    Train labels
There are:  155128   Test  labels


In [8]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size))) 

                        labels:  1.8 MiB
                documents_test:  1.3 MiB
                        Y_test:  1.2 MiB
                 Y_test_reduce:  1.2 MiB
               documents_train: 565.4 KiB
                       Y_train: 523.0 KiB
                    stop_words:  8.2 KiB
                          _iii:  1.4 KiB
                           _i5:  1.4 KiB
                    Sequential:  1.0 KiB


In [9]:
def pre_processing_matlab(conversation):
    # First pre-processing: Matlab
    conversation = conversation.replace('"','') 
    conversation = conversation.replace('.',' ') 
    conversation = conversation.replace("''",' ')
    conversation = conversation.lower()
    conversation = conversation.replace('[^\w\s]',' ') 
    
    # Second pre-processing: TMG

    # Remove punctuation as: !?&
    conversation = conversation.translate(str.maketrans('', '', string.punctuation))
    """
    # Remove numerbs 
    conversation = re.sub(r'\b[0-9]+\b', '', conversation)#and alphanumerics too : re.sub('[0-9]+', '', i)
    """
    # Remove  alphanumerics
    conversation = re.sub('[0-9]+', '', conversation)   
    
    # Tokenize and remove SW
    tokens = word_tokenize(conversation)
    conversation = [i for i in tokens if not i in stop_words]
    
    # Applying Potter Steamming
    #conversation = [porter.stem(i) for i in conversation ]
    
    # Applying Lemmatization
    conversation = [wordnet_lemmatizer.lemmatize(i, pos="v") for i in conversation ]
    
    conversation = " ".join(conversation)
    
    return conversation


In [10]:
conversation = "MY name is  dani, i am running with my thoughts, i am furious"
aux = pre_processing_matlab(conversation)
print(aux)

name dani run thoughts furious


Processing Train documents

In [11]:
def pre_processing_dani(documents,labels):
    docs = []
    Y = []
    empty_docs = 0
    for id,conversation in enumerate(documents):
        clean_conversation = pre_processing_matlab(conversation)
        if len (clean_conversation) == 0:
            empty_docs += 1
        else:
            docs.append(clean_conversation)
            Y.append(labels[id])
    print("Empty Documetns: ",empty_docs)
    return docs,Y

In [12]:
print("Before: ",documents_train[0])
start = time.time()
doc_train,new_Y_train = pre_processing_dani(documents_train,Y_train)
print("Train Documents, There are :",len(doc_train),": ", doc_train[0]," ",len(new_Y_train))
end = time.time()
print("Time is: ",end - start)

Before:  hi asl hey checking for a girl? So i am 23 thirty I have fantastic vids are going to head over to goto surprisee21 tk (replace space with  ) 
Empty Documetns:  174
Train Documents, There are : 66753 :  hi asl hey check girl thirty fantastic vids go head goto surprisee tk replace space   66753
Time is:  1740.3499557971954


Processing Test documents

In [13]:
print("Before: ",documents_test[0])
start = time.time()
doc_test,new_Y_test = pre_processing_dani(documents_test,Y_test)
print("Test Documents, There are :",len(doc_test),": ", doc_test[0]," ",len(new_Y_test))
end = time.time()
print("Time is: ",end - start)


Before:  wirepair: should we all do the ie way with a different constructor? XDomainRequest or what it's called Hello all, I have a quick question about the history API is there something like the same domain policy on it? 
Empty Documetns:  305
Test Documents, There are : 154823 :  wirepair ie way different constructor xdomainrequest call hello quick question history api something like domain policy   154823
Time is:  716.7278778553009


In [14]:
#"""
Y_train = np.asarray(new_Y_train)
Y_test = np.asarray(new_Y_test)

posi_train = np.argwhere(Y_train==1)
posi_test  = np.argwhere(Y_test==1)
nega_train = np.argwhere(Y_train==0)
nega_test  = np.argwhere(Y_test==0)
print("Predadors: ",len(posi_train)," No Predadors: ", len(nega_train))
print("Predadors: ",len(posi_test)," No Predadors: ", len(nega_test))
#"""print("Doc Train:",len(doc_train)," Labels Train: ",len(new_Y_train))
print("Doc Test :",len(doc_test)," Labels Test : ",len(new_Y_test))



Predadors:  1958  No Predadors:  64795
Predadors:  3666  No Predadors:  151157
Doc Test : 154823  Labels Test :  154823


In [15]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size))) 

                        labels:  1.8 MiB
                documents_test:  1.3 MiB
                      doc_test:  1.3 MiB
                    new_Y_test:  1.3 MiB
                 Y_test_reduce:  1.2 MiB
                        Y_test:  1.2 MiB
               documents_train: 565.4 KiB
                     doc_train: 565.4 KiB
                   new_Y_train: 565.4 KiB
                       Y_train: 521.6 KiB


In [16]:
del labels,new_Y_train,new_Y_test
gc.collect()

0

n-grms: Only from doc train

In [17]:
"""
dicc_ngrms = {}
sentence = 'this is a foo bar sentences and i want to ngramize it this is'

n = 1
sixgrams = ngrams(sentence.split(), n)

for grams in sixgrams:
    aux = " ".join(list(grams))
    if aux in dicc_ngrms:
        dicc_ngrms[aux] += 1
    else:
        dicc_ngrms[aux] = 0

min_df = 2        
n_grmas_list = []
for k,v  in dicc_ngrms.items():
    if v >= min_df:
        n_grmas_list.append(k)
print(dicc_ngrms)  
"""

'\ndicc_ngrms = {}\nsentence = \'this is a foo bar sentences and i want to ngramize it this is\'\n\nn = 1\nsixgrams = ngrams(sentence.split(), n)\n\nfor grams in sixgrams:\n    aux = " ".join(list(grams))\n    if aux in dicc_ngrms:\n        dicc_ngrms[aux] += 1\n    else:\n        dicc_ngrms[aux] = 0\n\nmin_df = 2        \nn_grmas_list = []\nfor k,v  in dicc_ngrms.items():\n    if v >= min_df:\n        n_grmas_list.append(k)\nprint(dicc_ngrms)  \n'

In [18]:
"""
# Put toguether the hold train text
all_text = ""
for i in doc_train:
    all_text += i + " "
    
n = 1
dicc_ngrms = {}

grams_text = ngrams(all_text.split(), n)
for grams in grams_text:
    aux = " ".join(list(grams))
    if aux in dicc_ngrms:
        dicc_ngrms[aux] += 1
    else:
        dicc_ngrms[aux] = 0

min_df = 5      
n_grmas_list = []
for k,v  in dicc_ngrms.items():
    if v >= min_df:
        n_grmas_list.append(k)
        
n_grmas_list.sort()
print(len(n_grmas_list))    
"""

'\n# Put toguether the hold train text\nall_text = ""\nfor i in doc_train:\n    all_text += i + " "\n    \nn = 1\ndicc_ngrms = {}\n\ngrams_text = ngrams(all_text.split(), n)\nfor grams in grams_text:\n    aux = " ".join(list(grams))\n    if aux in dicc_ngrms:\n        dicc_ngrms[aux] += 1\n    else:\n        dicc_ngrms[aux] = 0\n\nmin_df = 5      \nn_grmas_list = []\nfor k,v  in dicc_ngrms.items():\n    if v >= min_df:\n        n_grmas_list.append(k)\n        \nn_grmas_list.sort()\nprint(len(n_grmas_list))    \n'

In [19]:
#n_grmas_list

In [20]:
# Dicctionary is only create from train documents, like in real life
"""
vocabulary_python = {}
for document in doc_train:
    document = document.lower()
    document = document.split()
    for term in document:
        if term in vocabulary_python:
            vocabulary_python[term] += 1
        else:
            vocabulary_python[term] = 0
            
"""

'\nvocabulary_python = {}\nfor document in doc_train:\n    document = document.lower()\n    document = document.split()\n    for term in document:\n        if term in vocabulary_python:\n            vocabulary_python[term] += 1\n        else:\n            vocabulary_python[term] = 0\n            \n'

In [21]:
"""
v = []
for key in vocabulary_python:
    if vocabulary_python[key] >= 5:
        #print(key, '->', vocabulary_python[key])
        #del vocabulary_python[key]
        v.append(key)
v.sort()
print(len(v))
"""

"\nv = []\nfor key in vocabulary_python:\n    if vocabulary_python[key] >= 5:\n        #print(key, '->', vocabulary_python[key])\n        #del vocabulary_python[key]\n        v.append(key)\nv.sort()\nprint(len(v))\n"

In [22]:
#print(v)

In [23]:
from scipy import sparse
start = time.time()

#Note, that lower case ( ) in Vectorizer is by default

# Our Vocabulary : 26784 words
#vectorizer = CountVectorizer(vocabulary=n_grmas_list,dtype=np.int64) 

word_gram = 1
vectorizer = CountVectorizer(dtype=np.int64,min_df=5,strip_accents="ascii") 

# 3-char n gram
#word_gram = 2
#vectorizer = CountVectorizer(analyzer='char', ngram_range=(word_gram, word_gram),dtype=np.int64,min_df=5)



X = vectorizer.fit_transform(doc_train)
X_train = X.toarray()#sparse.csr_matrix(X)#
print(X_train.shape)
end = time.time()
print("Time is: ",end - start)
#print(vectorizer.get_feature_names())

(66753, 20975)
Time is:  19.06559109687805


In [24]:
print(X[6542])

  (0, 7651)	2


In [25]:
DicC = vectorizer.get_feature_names()
print(len(DicC)," ",DicC[765])

20975   ano


In [26]:
print(DicC)

['aTM', 'aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaah', 'aaaah', 'aaabeeceeaebcbdeab', 'aaah', 'aaand', 'aaargh', 'aabaadeeef', 'aabb', 'aabdbdfbdfae', 'aac', 'aacc', 'aacccebfcfeee', 'aaddbfecafba', 'aadfbbcdfdbee', 'aadil', 'aae', 'aaedefafe', 'aaeua', 'aafcefacd', 'aah', 'aahh', 'aahhh', 'aai', 'aakb', 'aal', 'aampe', 'aan', 'aand', 'aao', 'aap', 'aargh', 'aaron', 'aaroncampbell', 'aaronlev', 'aaronpk', 'aasl', 'aau', 'ab', 'abaeacabcacac', 'abafeebecdfcfae', 'abandon', 'abarth', 'abarths', 'abbeaef', 'abbffbdfaba', 'abbr', 'abbreviate', 'abbreviation', 'abbreviations', 'abbs', 'abby', 'abc', 'abdfedacbfebea', 'abe', 'abebcacffbeefbbcacabf', 'abecfaccdd', 'abeedbe', 'abend', 'aber', 'abernier', 'abfaadcccbb', 'abfaaedbeaffc', 'abfaaedbeaffcs', 'abg', 'abhi', 'abhishek', 'abi', 'abide', 'abilities', 'ability', 'abit', 'able', 'abnf', 'abnormal', 'abomination', 'aboodman', 'abort', 'abortion', 'abot', 'abotu', 'abou', 'aboutblank', 'aboutconfig', 'aboutquot', 'abouts', 'abovequot', 'aboy', 'a

In [27]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size))) 

                       X_train: 10.4 GiB
                documents_test:  1.3 MiB
                      doc_test:  1.3 MiB
                 Y_test_reduce:  1.2 MiB
                        Y_test:  1.2 MiB
               documents_train: 565.4 KiB
                     doc_train: 565.4 KiB
                       Y_train: 521.6 KiB
                          DicC: 173.9 KiB
                    stop_words:  8.2 KiB


In [28]:
#del vocabulary_python,v,vectorizer,X
#gc.collect()

In [31]:
start = time.time()
#vectorizer = CountVectorizer(analyzer='char',vocabulary=DicC,dtype=np.int64,ngram_range=(word_gram, word_gram)) 
vectorizer = CountVectorizer(vocabulary=DicC,dtype=np.int64,strip_accents="ascii") 
X = vectorizer.fit_transform(doc_test)
print(X.shape)
end = time.time()
print("Time is: ",end - start)
X_test = X.toarray()

(154823, 20975)
Time is:  7.341437578201294


In [ ]:
"""
cadena = [" hi i am dani yet seem"]
vectorizer = CountVectorizer(vocabulary=DicC,dtype=np.int64,ngram_range=(word_gram, word_gram)) 
X = vectorizer.fit_transform(cadena)
print(X)
"""

In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size))) 

In [ ]:
#del documents_train,documents_test
#gc.collect()

In [32]:
def sklearn_mesures(Y_pred,Y_test,mesure):
    a  = accuracy_score(Y_test, Y_pred, normalize=True)
    p  = precision_score(Y_test, Y_pred , average=mesure)
    r  = recall_score(Y_test, Y_pred , average=mesure)
    f1_c1 = f1_score(Y_test, Y_pred , average=mesure,pos_label=0)
    f1_c2 = f1_score(Y_test, Y_pred , average=mesure,pos_label=1)
    return [f1_c1,f1_c2]#[a,p,r,f1_c1,f1_c2]



In [33]:
def f1_matlab(predict,true,c):
    # This function outputs the F1 per class , this from Matlab
    erx = np.sign(predict) - np.sign(true)
    ofc = np.argwhere(np.sign(true)==c) 
    pfc = np.argwhere(np.sign(predict)==c)

    p = (len(np.argwhere(erx[pfc]==0))/len(pfc))
    r = (len(np.argwhere(erx[ofc]==0))/len(ofc))
    
    tmp = (r + p)

    f = ((2 * p * r) / tmp) * 100
    return [f]

In [34]:
# Making the predicction, and setting threshold to 0.5
def predicction(Y_pred, Y_test):
    Y_pred_bool = []
    Y_pred = np.where(Y_pred > 0.5, 1, 0) 
    for i in Y_pred:
        Y_pred_bool.append(i[0]) 
    Y_pred_bool = np.asarray(Y_pred_bool) 

    non_predadors  = 0
    predadors      = 0
    for v in Y_pred_bool:
        if v == 0:
            non_predadors += 1
        else:
            predadors += 1
        
    if (non_predadors == 0 or predadors == 0):
        print("Empty Label!!!")
        print("Predicctions with sklearn")
        print(sklearn_mesures(Y_pred,Y_test,"binary"))
        return [1,0]
    else:
        
        # Predictions con sklearn mesures and matlab function
        c1_f1 = f1_matlab(Y_pred_bool,Y_test,0)
        c2_f1 = f1_matlab(Y_pred_bool,Y_test,1)
        print("Predicctions with sklearn")
        print(sklearn_mesures(Y_pred,Y_test,"binary"))
        print("Predicctions with Matlab")
        print(c1_f1," ",c2_f1)
        print("Confusion Matrix")
        print(confusion_matrix(Y_pred_bool, Y_test))
    return [c1_f1[0],c2_f1[0]]

In [35]:
def csa (X,Y):
    num_docs = X.shape[0] # number of trainning docs
    num_terms = X.shape[1]# number of termos
    num_class = 2
    TR = np.zeros((num_terms, num_class))
    DL = np.sum(X, axis=1) # sum each elemet row, wich is each df element of a document
    DL[DL <1 ] = 1 # Those documents with not terms in dicc, are empty . We put one for avoid div / 0.
    
   
    # Non - Normalization 0'
    for j in range (0,num_class): # j = [0;1]
        docs = np.where(Y == j)[0] 
        for i in range(0,num_terms):
            tf_v = X[docs,i] # tf_v is tern frecuency of each term
            TR[i][j] = sum(np.log2(1 + tf_v / DL [docs]))        
            
            if np.isnan(TR[i][j]):
                TR[i][j] = 0
    #TR = np.nan_to_num(TR)
    print("PRB: (",TR.shape[0],",",TR.shape[1],")")

    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    n1[n1 < 1] = 1
    Tik = TR / n1
    #Tik = np.nan_to_num(Tik)

    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        suma = sum(Tik[i])
        if suma == 0:
            suma = 1
        Tik[i] = Tik[i] / suma # Tik[i] = Tik[i] / sum(TR[i])

    print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    
    # Document Representation
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X[i] != 0)[0] # extrac idxs =! 0
        a = X[i][termos] / DL[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR[i] = sum(np.multiply(a,Tik[termos])) # alpha (a)* tik; alpha is the df of each term
    # Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"    return Tik
    
    

    return DR,Tik

In [36]:
start = time.time()
DR,Tik = csa (X_train,Y_train)
end = time.time()
print("DR: (",DR.shape[0],",",DR.shape[1],")")
print("Time: ",end-start)

PRB: ( 20975 , 2 )
Tik: ( 20975 , 2 )
DR: ( 66753 , 2 )
Time:  393.4167182445526


In [37]:
def csa_test (XT,Tik): 
    # Document Test Representation
    num_class = 2
    DLT = np.sum(XT, axis=1)
    DLT[DLT <1 ] = 1
    num_docs = XT.shape[0]
    DRT = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(XT[i] != 0)[0] # extrac idxs =! 0
        a = XT[i][termos] / DLT[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DRT[i] = sum(np.multiply(a,Tik[termos]))
    return DRT

In [38]:
start = time.time()
DRT = csa_test (X_test,Tik)
end = time.time()
print("DRT: (",DRT.shape[0],",",DRT.shape[1],")")
print("Time: ",end-start)

DRT: ( 154823 , 2 )
Time:  14.989444732666016


In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size))) 

In [ ]:
del X_train,X_test
gc.collect()

In [ ]:
del model
gc.collect()
K.clear_session()

In [39]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [40]:
#Create NN model
num_class = 2
model = Sequential()
model.add(LSTM(256, input_shape=(num_class, 1)))
#model.add(Bidirectional(LSTM(64, input_shape=(num_class, 1))))
model.add(Dense(1, activation='sigmoid'))
#print(model.summary())

In [41]:
#opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False,decay=1e-14)#adam(decay=1e-14)
opt = keras.optimizers.Adam(decay=1e-14)#adam(decay=1e-14)
#opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [42]:
A = np.reshape(DR, (DR.shape[0],  DR.shape[1],1))
B = np.reshape(DRT, (DRT.shape[0],DRT.shape[1],1))
model.fit(A, Y_train, epochs=30, batch_size=10, validation_data=(B, Y_test),verbose=1)




Train on 66753 samples, validate on 154823 samples
Epoch 1/30





66753/66753 [==============================] - 123s 2ms/step - loss: 0.0525 - acc: 0.9800 - val_loss: 0.0498 - val_acc: 0.9810
Epoch 2/30
66753/66753 [==============================] - 119s 2ms/step - loss: 0.0434 - acc: 0.9831 - val_loss: 0.0538 - val_acc: 0.9806
Epoch 3/30
66753/66753 [==============================] - 123s 2ms/step - loss: 0.0433 - acc: 0.9834 - val_loss: 0.0503 - val_acc: 0.9805
Epoch 4/30
66753/66753 [==============================] - 126s 2ms/step - loss: 0.0431 - acc: 0.9831 - val_loss: 0.0491 - val_acc: 0.9810
Epoch 5/30
66753/66753 [==============================] - 123s 2ms/step - loss: 0.0431 - acc: 0.9832 - val_loss: 0.0503 - val_acc: 0.9813
Epoch 6/30
66753/66753 [==============================] - 123s 2ms/step - loss: 0.0430 - acc: 0.9833 - val_loss: 0.0479 - val_acc: 0.9822
Epoch 7/30
66753/66753 [==============================] - 125s 2ms/step - loss: 0.0429 - acc: 0.9832 - val_loss: 

In [43]:
model.evaluate(B, Y_test)
Y_pred = model.predict(B, batch_size=10, verbose=0)
predicction(Y_pred, Y_test)

154823/154823 [==============================] - 7s 45us/step
Predicctions with sklearn
[0.9908306096700807, 0.6386532689547175]
Predicctions with Matlab
[99.08306096700808]   [63.86532689547175]
Confusion Matrix
[[149607   1219]
 [  1550   2447]]


[99.08306096700808, 63.86532689547175]

Calculando el error de los predadores .

In [44]:
######
# This is the real f1 mesure , taking into account the error
#
#
#
# 
Y_test_fake = Y_test
lost_predadors = np.zeros(3737-3666)
lost_no_predadors = np.zeros(155128-151157)
Y_test_fake_lost_predadors = np.concatenate((Y_test_fake, lost_predadors))
Y_test_fake_lost_predadors = np.concatenate((Y_test_fake_lost_predadors, lost_no_predadors))
posi_test  = np.argwhere(Y_test_fake_lost_predadors==1)
print("posi_test ",len(posi_test))

Y_pred_fake = Y_pred

miss_predadors = np.ones((3737-3666,1))
Y_pred_fake_miss_predadors = np.concatenate((Y_pred_fake, miss_predadors))
loss_no_pr = np.zeros((155128-151157,1))
Y_pred_fake_miss_predadors = np.concatenate((Y_pred_fake_miss_predadors, loss_no_pr))
Y_pred_fake_miss_predadors = np.where(Y_pred_fake_miss_predadors > 0.5, 1, 0) 

posi_test  = np.argwhere(Y_pred_fake_miss_predadors==1)
print("posi_test ",len(posi_test))



print("after: ")
predicction(Y_pred_fake_miss_predadors, Y_test_fake_lost_predadors)

#
#
#
################################################


posi_test  3666
posi_test  4068
after: 
Predicctions with sklearn
[0.9908385914656963, 0.6327902767002845]
Predicctions with Matlab
[99.08385914656964]   [63.279027670028455]
Confusion Matrix
[[153578   1219]
 [  1621   2447]]


[99.08385914656964, 63.279027670028455]

In [ ]:
Predicctions with sklearn
[0.9908306096700807, 0.6386532689547175]
Predicctions with Matlab
[99.08306096700808]   [63.86532689547175]
Confusion Matrix
[[149607   1219]
 [  1550   2447]]

[99.08306096700808, 63.86532689547175]


error
154822/154822 [==============================] - 7s 42us/step
Predicctions with sklearn
[0.9905896635564239, 0.6526264234112894]
Predicctions with Matlab
[99.05896635564238]   [65.26264234112894]
Confusion Matrix
[[149320   1001]
 [  1836   2665]]

[99.05896635564238, 65.26264234112894]

We predict 20 different .xml files containing [0.01%,0.06%,0.11%,0.16%,0.21%,0.26%,0.31%,0.36%,0.41%,0.46%,0.51%,0.56%,0.61%,0.66%,0.71%,0.76%,0.81%,0.86%,0.91%,1%] of tha information.

In [ ]:
# The order of the Labels are others, remeber Matlab re order the data
Y_test  = 'pan12_test/predadors.txt'
f = open(Y_test, "r") # Switch from Train to Test 
predadors = []

for x in f:
    x = x.strip()
    predadors.append(x)
    #print(x)
predadors = np.unique(predadors)
print("Unique predadors: ",len(predadors))

In [ ]:
import xml.etree.ElementTree as ET
from collections import Counter

In [ ]:
# Check witch file pass the filter, return (filter_docuemnts,labels)
def filtering(file_name):
    #
    Y_test  = 'pan12_test/predadors.txt'
    f = open(Y_test, "r") # Switch from Train to Test 
    predadors = []

    for x in f:
        x = x.strip()
        predadors.append(x)
        #print(x)
    predadors = np.unique(predadors)
    #

    print("Arrive: ",file_name)
    tree = ET.parse(file_name ) 
    root = tree.getroot()
    
    start = time.time()
    i = 0
    id = 0
    j = 1
    labels = []
    conversations_dicc = {}
    all_unique_authors = []
    predadors_in_conve = 0
    for conversation in root.iter('conversation'):
        authors = []
        # For each conversation we count the number of authors and their interactions
        for author in conversation.iter('author'):
            authors.append(author.text)
        filtering = Counter(authors)
        # Flag: True conversations pass the filter
        taken = True
        
        #Just one author
        if len(filtering) < 2: 
            taken = False
        else:
            # Less of 6 interaction per author
            for f in filtering:
                if filtering[f] < 6: 
                    taken = False # Basta que uno interactue menos de 6 veces para invalidar la conversa
                    break
        # Conversations pass the filetering
        if taken == True:
            #print(Counter(authors))
            i += 1
            tmp = np.unique(authors)
            # Counting the number of unique authors, just to know (not usefull)
            for t in tmp:
                all_unique_authors.append(t)
            # Making new Labels: if some author is in Predador, then 1
            if len(np.intersect1d(predadors,tmp)) > 0:
                labels.append(1)   
                predadors_in_conve += 1
            else:
                labels.append(0)  
                
            # Only if the conversations pass the filtering, we save it 
            ##
            text = ''
            for message in conversation.iter('message'):
                if message[2].text: 
                    # message[0] is author, message[1] is time, message[2] is text
                    msg =  message[2].text + ' '
                else:
                     msg = ' ' #Somo message are empty
                text += msg
            # Saving each conversation in a diccionary
            conversations_dicc[id] = text
            ##
            id += 1
            #print(j)
            #print(text)
            #print("-------------------------------")
        j += 1 # Num of cove with out filter
        
    all_unique_authors = np.unique(all_unique_authors)
    labels = np.asarray(labels)
    end = time.time()
    print("Total time: ",end-start)
    print("Number of Conver: ",i)
    print("Number of Users: ",len(all_unique_authors))
    print("Predadors in Conversations: ",predadors_in_conve)
    print("Number of Predadors: ",len(np.intersect1d(predadors,all_unique_authors)))
    print("Number of Saved Conver: ",len(conversations_dicc))
    return conversations_dicc,labels





In [ ]:
#file_name = "OficialData/xml/0.11.xml"
#_c,_l = filtering(file_name)

In [ ]:
retains = [0.01,0.06,0.11,0.16,0.21,0.26,0.31,0.36,0.41,0.46,0.51,0.56,0.61,0.66,0.71,0.76,0.81,0.86,0.91,1]
directory = "OficialData/xml"
doc_predictions = []
times = []
for r in retains:

    start = time.time() 
    
    print(directory+"/"+str(r)+".xml")
    file_name = directory+"/"+str(r)+".xml"
    #_c,_l = filtering(file_name)
    # Collecting the data that pass the filtering process
    non_process_doc, labels = filtering(file_name)
    predadors = np.argwhere(labels==1)
    non_predadors  = np.argwhere(labels==0)
    print("Predadors: ",len(predadors)," No Predadors: ", len(non_predadors))
    # Converting text into n-grams
    #sd_grams = n_grams_filtering(non_process_doc)
    #
    
    # Processing the set of documents
    empty_docs = 0
    doc_test = []
    new_Y_test = []
    for id,conversation in enumerate(non_process_doc):
        clean = pre_processing(non_process_doc[conversation])
        if len (clean) == 0:
            # We do not add the empty doc, and we delete that label
            empty_docs += 1
        else:
            doc_test.append(clean)
            new_Y_test.append(labels[id])
    print("Empty docs: ",empty_docs)
    new_Y_test = np.asarray(new_Y_test)
    print("Docs: ",len(doc_test)," Labels: ",new_Y_test.shape)
    #
    
    
    # Transforming the documents to a matrix
    #"""
    X = vectorizer.fit_transform(doc_test)
    X_test = X.toarray()
    print(X_test.shape)  
    #"""

    # Applying the algorithm
    DRTS = csa_test (X_test,Tik)
    print("DRT: (",DRTS.shape[0],",",DRTS.shape[1],")")

    # Testing the test examples in the NN.
    model.evaluate(DRTS, new_Y_test)
    Y_pred = model.predict(DRTS, batch_size=10, verbose=0)    
    end = time.time() 
    # Making predictions
    p = predicction(Y_pred, new_Y_test)
    doc_predictions.append(p)
    times.append(end - start)
    print("Time is: ",end - start)
    
    del X_test,DRTS,non_process_doc,labels
    gc.collect()
    print("-"*50)
    
    



In [ ]:
# F1
for id,d in enumerate(doc_predictions):
    #print(id+1," ",d[1])
    print(d[1])

In [ ]:
# Times
for t in enumerate(times):
    print(t[1])

In [ ]:
import matplotlib.pyplot as plt

def pie_plot(predador,no_predador,word,labels):
    sizes = [predador,no_predador]
    colors = ['lightskyblue', 'lightcoral']
    explode = (0, 0)  # explode a slice if required

    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True)

    #draw a circle at the center of pie to make it look like a donut
    centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1.25)
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.title( word, bbox={'facecolor':'0.8', 'pad':5})

    # Set aspect ratio to be equal so that pie is drawn as a circle.
    plt.axis('equal')
    plt.show() 

In [ ]:
DicC = np.asarray(DicC)
DicC = np.reshape(DicC,(len(DicC),1))
print(DicC.shape)
words_representation = np.hstack((Tik,DicC))
print("DicC: ",len(DicC)," Tik: ",Tik.shape," words_representation: ",words_representation.shape)



In [ ]:
predador_wr_sorted = words_representation[np.argsort(words_representation[:, 1])][::-1]
no_predador_wr_sorted = words_representation[np.argsort(words_representation[:, 0])][::-1]

In [ ]:
print("More significant words in Predador class: ")
print(predador_wr_sorted[:100])
print("More significant words in No - Predador class: ")
print(no_predador_wr_sorted[:100])

In [ ]:
labels = 'No - Predador', 'Predador'
index = 0
print(predador_wr_sorted[index])
predador = float(predador_wr_sorted[index][0])
no_predador = float(predador_wr_sorted[index][1])
word = predador_wr_sorted[index][2]
pie_plot(predador,no_predador,word,labels)